In [1]:
library("lme4")
library("ggplot2")
library("dplyr")
library(MASS)
library("BayesFactor")
library("car")
library("scales")
library("lmerTest")
library("MuMIn")
library("plyr")
library("knitr")
library("rstatix")
library("ggpubr")
library("multcompView")
library("multcomp")
library("lsmeans")
library("rcompanion")
require(nlme)
library("agricolae")

Loading required package: Matrix


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: coda

************
Welcome to BayesFactor 0.9.12-4.2. If you have questions, please contact Richard Morey (richarddmorey@gmail.com).

Type BFManual() to open the manual.
************

Loading required package: carData

Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

   

In [2]:
#Load the data => Here, we are filtering the cases in which subjects had a MuIFI of 0
FinalPath<-'../ToInput/R_HumMod_MS17_notemp.txt'
data <- read.delim(FinalPath, sep="\t", header=T, stringsAsFactors=F)
data <- data %>% filter(!Amplitude==0) 
dataTest <- data
data$lCV <- log(data$CV)
NArow <- which(is.na(data$lCV))
data <- data %>% filter(!is.na(lCV))

In [3]:
##Source for "SummarySE": http://www.cookbook-r.com/Manipulating_data/Summarizing_data/
##
##
## Gives count, mean, standard deviation, standard error of the mean, and confidence interval (default 95%).
##   data: a data frame.
##   measurevar: the name of a column that contains the variable to be summariezed
##   groupvars: a vector containing names of columns that contain grouping variables
##   na.rm: a boolean that indicates whether to ignore NA's
##   conf.interval: the percent range of the confidence interval (default is 95%)
summarySE <- function(data=NULL, measurevar, groupvars=NULL, na.rm=FALSE,
                      conf.interval=.95, .drop=TRUE) {
    library(plyr)

    # New version of length which can handle NA's: if na.rm==T, don't count them
    length2 <- function (x, na.rm=FALSE) {
        if (na.rm) sum(!is.na(x))
        else       length(x)
    }

    # This does the summary. For each group's data frame, return a vector with
    # N, mean, and sd
    datac <- ddply(data, groupvars, .drop=.drop,
      .fun = function(xx, col) {
        c(N    = length2(xx[[col]], na.rm=na.rm),
          mean = mean   (xx[[col]], na.rm=na.rm),
          sd   = sd     (xx[[col]], na.rm=na.rm)
        )
      },
      measurevar
    )

    # Rename the "mean" column    
    datac <- rename(datac, c("mean" = measurevar))

    datac$se <- datac$sd / sqrt(datac$N)  # Calculate standard error of the mean

    # Confidence interval multiplier for standard error
    # Calculate t-statistic for confidence interval: 
    # e.g., if conf.interval is .95, use .975 (above/below), and use df=N-1
    ciMult <- qt(conf.interval/2 + .5, datac$N-1)
    datac$ci <- datac$se * ciMult

    return(datac)
}

In [4]:
# Multiple plot function: source: http://www.cookbook-r.com/Graphs/Multiple_graphs_on_one_page_(ggplot2)/
#
#
# ggplot objects can be passed in ..., or to plotlist (as a list of ggplot objects)
# - cols:   Number of columns in layout
# - layout: A matrix specifying the layout. If present, 'cols' is ignored.
#
# If the layout is something like matrix(c(1,2,3,3), nrow=2, byrow=TRUE),
# then plot 1 will go in the upper left, 2 will go in the upper right, and
# 3 will go all the way across the bottom.
#
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  library(grid)

  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)

  numPlots = length(plots)

  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
  }

 if (numPlots==1) {
    print(plots[[1]])

  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))

    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))

      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

# Extract First 5 games data

In [5]:
#Getting means for games 1-5
Data1_5 <- data %>% filter(Agent=="Human")
n_games <- 15
for(i in 6:15){
    Data1_5 <- Data1_5 %>% filter(GameNb!=i)
}
table(Data1_5$Agent, Data1_5$GameSpeed)

DataM1_5 <- ddply(Data1_5, c("Subject","Agent","GameSpeed"), summarize,  Amplitude=mean(Amplitude),Periodicity=mean(Periodicity),Entropy=mean(Entropy),lCV=mean(lCV))
DataM1_5 <- DataM1_5[order(DataM1_5$Subject),] # order by subject
nb_subjs1<-nrow(DataM1_5)
nb_subjs1
#nrow(DataM1_5)
head(DataM1_5,10)

       
        MMM MSM SMS SSS
  Human  80  92  95  84

[1] 77

,Subject,Agent,GameSpeed,Amplitude,Periodicity,Entropy,lCV
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,A10GBO6PU2EI0K,Human,MSM,0.19151277,784.0,3.246518,-1.2202040
2,A12WDL4PB927P6,Human,SMS,0.14389429,1084.8,3.589319,-0.7473337
3,A12ZO39VZ879ZS,Human,SMS,0.11915947,1180.8,3.394718,-1.2443726
4,A15J42M6EC7Y1K,Human,SSS,0.07435005,1193.6,3.665702,-1.2107393
5,A16335MOISDG1F,Human,MSM,0.14269687,640.0,3.172987,-0.9372757
6,A19M1TGTL9HKHB,Human,SSS,0.12906968,1305.6,3.491844,-1.0682865
7,A1AJ2G7JXXJ8UJ,Human,MMM,0.15628601,726.4,3.160044,-1.1044700
8,A1E8ZYOQSGKTP8,Human,MSM,0.38543002,992.0,3.398248,-0.2857093
9,A1EQAFAN71Z0SK,Human,SSS,0.18445878,1260.8,3.633584,-1.2598311


# Extract Middle 5 games data

In [6]:
#Getting means for games 6-10
Data6_10 <- data %>% filter(Agent=="Human")
n_games <- 15
for(i in 1:5){
    Data6_10 <- Data6_10 %>% filter(GameNb!=i)
}
for(i in 11:15){
    Data6_10 <- Data6_10 %>% filter(GameNb!=i)
}
table(Data6_10$Agent, Data6_10$GameSpeed)

DataM6_10 <- ddply(Data6_10, c("Subject","Agent","GameSpeed"), summarize,  Amplitude=mean(Amplitude),Periodicity=mean(Periodicity),Entropy=mean(Entropy),lCV=mean(lCV))
DataM6_10 <- DataM6_10[order(DataM6_10$Subject),] # order by subject
nb_subjs<-nrow(DataM6_10)
nb_subjs
head(DataM6_10,10)

       
        MMM MSM SMS SSS
  Human  95  97  90  93

[1] 78

,Subject,Agent,GameSpeed,Amplitude,Periodicity,Entropy,lCV
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,A10GBO6PU2EI0K,Human,MSM,0.06550943,1219.2,3.332735,-1.3783059
2,A12WDL4PB927P6,Human,SMS,0.19825693,904.0,3.086130,-0.7575121
3,A12ZO39VZ879ZS,Human,SMS,0.23277921,748.8,3.018638,-1.3199233
4,A15J42M6EC7Y1K,Human,SSS,0.09187504,1347.2,3.581618,-1.2493415
5,A16335MOISDG1F,Human,MSM,0.20426644,1267.2,3.402664,-1.5323308
6,A19M1TGTL9HKHB,Human,SSS,0.10614098,1456.0,3.373728,-1.2621086
7,A1AJ2G7JXXJ8UJ,Human,MMM,0.12836712,739.2,2.964896,-1.3636945
8,A1E8ZYOQSGKTP8,Human,MSM,0.23646547,1331.2,3.745417,-0.5282842
9,A1EQAFAN71Z0SK,Human,SSS,0.26503487,1212.8,3.296914,-1.3491235


In [7]:
#One way
for(aa in 1:nb_subjs){
    cur_Subject <- DataM6_10$Subject[aa]
    if(!(cur_Subject %in% DataM1_5$Subject)){
        DataM6_10 <- DataM6_10[-aa,]
    }
}
nb_Ags2<-nrow(DataM6_10)
nb_Ags2
#Second way
for(aa in 1:nb_subjs1){
    cur_Subject <- DataM1_5$Subject[aa]
    if(!(cur_Subject %in% DataM6_10$Subject)){
        DataM1_5 <- DataM1_5[-aa,]
    }
}
nb_Ags1<-nrow(DataM1_5)
nb_Ags1


#filter slow and medium
Start_Slow <- DataM1_5 %>% filter(!GameSpeed=='MMM') %>% filter(!GameSpeed=='MSM') %>% filter(!GameSpeed=='SMS')
Start_Medium <- DataM1_5 %>% filter(!GameSpeed=='SSS') %>% filter(!GameSpeed=='SMS') %>% filter(!GameSpeed=='MSM')
Start_SlowT <- DataM1_5 %>% filter(!GameSpeed=='MMM') %>% filter(!GameSpeed=='MSM') %>% filter(!GameSpeed=='SSS')
Start_MediumT <- DataM1_5 %>% filter(!GameSpeed=='SSS') %>% filter(!GameSpeed=='SMS') %>% filter(!GameSpeed=='MMM')

#filter slow and medium
Mid_MSM <- DataM6_10 %>% filter(GameSpeed=='MSM')
Mid_SMS <- DataM6_10 %>% filter(GameSpeed=='SMS')
Mid_MMM <- DataM6_10 %>% filter(GameSpeed=='MMM')
Mid_SSS <- DataM6_10 %>% filter(GameSpeed=='SSS')

[1] 76

[1] 76

In [8]:
require(nlme)
require(multcomp)

# Periodicity

In [9]:
Hum_StMed <- Start_Medium$Periodicity
Ag_StMed <- Start_Medium$Subject
n1<-length(Hum_StMed)
n1
Hum_StSlow <- Start_Slow$Periodicity
Ag_StSlow <- Start_Slow$Subject
n2<-length(Hum_StSlow)
n2
Hum_StMedT <- Start_MediumT$Periodicity
Ag_StMedT <- Start_MediumT$Subject
n1b<-length(Hum_StMedT)
n1b
Hum_StSlowT <- Start_SlowT$Periodicity
Ag_StSlowT <- Start_SlowT$Subject
n2b<-length(Hum_StSlowT)
n2b
Hum_MidMed <- Mid_SMS$Periodicity
Ag_MidMed <- Mid_SMS$Subject
n3<-length(Hum_MidMed)
n3
Hum_MidSlow <- Mid_MSM$Periodicity
Ag_MidSlow <- Mid_MSM$Subject
n4<-length(Hum_MidSlow)
n4
Hum_MidPureM <- Mid_MMM$Periodicity
Ag_MidPureM <- Mid_MMM$Subject
n5<-length(Hum_MidPureM)
n5
Hum_MidPureS <- Mid_SSS$Periodicity
Ag_MidPureS <- Mid_SSS$Subject
n6<-length(Hum_MidPureS)
n6
n_Total<-n1+n2+n1b+n2b+n3+n4+n5+n6
n_Total

[1] 18

[1] 19

[1] 20

[1] 19

[1] 19

[1] 20

[1] 18

[1] 19

[1] 152

In [10]:
Speed <- c(rep("Medium",n1b),rep("Slow",n2b),rep("Medium",n1),rep("Slow",n2),rep("Medium",n3),rep("Slow",n4),rep("Medium",n5),rep("Slow",n6))
Subject <- c(Ag_StMedT,Ag_StSlowT,Ag_StMed,Ag_StSlow,Ag_MidMed,Ag_MidSlow,Ag_MidPureM,Ag_MidPureS)
Periodicity <- c(Hum_StMedT,Hum_StSlowT,Hum_StMed,Hum_StSlow,Hum_MidMed,Hum_MidSlow,Hum_MidPureM,Hum_MidPureS)
Phase <- c(rep("StartT",n1b),rep("StartT",n2b),rep("StartNoT",n1),rep("StartNoT",n2),rep("MidT",n3),rep("MidT",n4),rep("MidNoT",n5),rep("MidNoT",n6))
length(Speed)
ToPlot_Per <- data.frame(Subject,Speed,Phase,Periodicity)
#ToPlot_Per$Speed <- reorder(ToPlot_Per$Speed,desc(ToPlot_Per$Speed))
head(ToPlot_Per)

[1] 152

,Subject,Speed,Phase,Periodicity
,<fct>,<fct>,<fct>,<dbl>
1,A10GBO6PU2EI0K,Medium,StartT,784.0
2,A16335MOISDG1F,Medium,StartT,640.0
3,A1E8ZYOQSGKTP8,Medium,StartT,992.0
4,A1YSBD5XW5UCR0,Medium,StartT,636.8
5,A2E609OEGN4D6H,Medium,StartT,692.0
6,A2GO3SYV3X3KE,Medium,StartT,784.0


In [11]:
ToPlot_Per %>% group_by(Speed,Phase) %>% shapiro_test(Periodicity)

Speed,Phase,variable,statistic,p
<fct>,<fct>,<chr>,<dbl>,<dbl>
Medium,MidNoT,Periodicity,0.8723401,0.0194485488
Medium,MidT,Periodicity,0.9617128,0.6065525051
Medium,StartNoT,Periodicity,0.9523278,0.4626940058
Medium,StartT,Periodicity,0.7590720,0.0002252181
Slow,MidNoT,Periodicity,0.9483836,0.3708185391
Slow,MidT,Periodicity,0.8518313,0.0057213927
Slow,StartNoT,Periodicity,0.9290110,0.1660696071
Slow,StartT,Periodicity,0.8610388,0.0101985197


In [12]:
#A/ case Start & No Transfer
Per_data <- ToPlot_Per %>% filter(!Phase=='MidT') %>% filter(!Phase=='StartT')
Per_Start <- (Per_data %>% filter(Phase=='StartNoT'))$Periodicity
Per_Mid <- (Per_data %>% filter(Phase=='MidNoT'))$Periodicity
median(Per_Start)
median(Per_Mid)
print('Phase comparison')
MWa<-wilcox.test(Per_Start,Per_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

Per_Slow <- (Per_data %>% filter(Speed=='Slow'))$Periodicity
Per_Med <- (Per_data %>% filter(Speed=='Medium'))$Periodicity
median(Per_Slow)
median(Per_Med)
print('Speed comparison')
MWb<-wilcox.test(Per_Med,Per_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] 988.8

[1] 982.4

[1] "Phase comparison"



	Wilcoxon signed rank test with continuity correction

data:  Per_Start and Per_Mid
V = 239, p-value = 0.2162
alternative hypothesis: true location shift is not equal to 0


[1] -1.236708

[1] 1217.6

[1] 692.8

[1] "Speed comparison"



	Wilcoxon rank sum test with continuity correction

data:  Per_Med and Per_Slow
W = 26, p-value = 1.151e-12
alternative hypothesis: true location shift is not equal to 0


[1] -7.11118

In [17]:
#B/ case Start & Transfer
Per_data2 <- ToPlot_Per %>% filter(!Phase=='MidNoT') %>% filter(!Phase=='StartNoT')
Per_Start <- (Per_data2 %>% filter(Phase=='StartT'))$Periodicity
Per_Mid <- (Per_data2 %>% filter(Phase=='MidT'))$Periodicity
print('Phase comparison')
median(Per_Start)
median(Per_Mid)
MWa<-wilcox.test(Per_Start,Per_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

Per_Slow <- (Per_data2 %>% filter(Speed=='Slow'))$Periodicity
Per_Med <- (Per_data2 %>% filter(Speed=='Medium'))$Periodicity
print('Speed comparison')
median(Per_Slow)
median(Per_Med)
MWb<-wilcox.test(Per_Med,Per_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] 931.2

[1] 982.4


	Wilcoxon signed rank test with continuity correction

data:  Per_Start and Per_Mid
V = 243, p-value = 0.0655
alternative hypothesis: true location shift is not equal to 0


[1] -1.841862

[1] "Speed comparison"


[1] 1216

[1] 748.8


	Wilcoxon rank sum test with continuity correction

data:  Per_Med and Per_Slow
W = 95, p-value = 2.996e-11
alternative hypothesis: true location shift is not equal to 0


[1] -6.646776

In [18]:
#C) Transfer by comparison with No Transfer
Transfer_compare <- Per_data2 %>% filter(Phase=='MidT')
ToAdd_Transfer <- rep('Transfer',nrow(Transfer_compare))
Transfer_compare$Group <- ToAdd_Transfer
nrow(Transfer_compare)
NoTransfer_compare <- Per_data %>% filter(Phase=='MidNoT')
ToAdd_NoTransfer <- rep('No Transfer',nrow(NoTransfer_compare))
NoTransfer_compare$Group <- ToAdd_NoTransfer
#nrow(NoTransfer_compare)
#Compare_T <- rbind(Transfer_compare,NoTransfer_compare)
#nrow(Compare_T)

Per_Transfer <- Transfer_compare$Periodicity
Per_NoTransfer <- NoTransfer_compare$Periodicity
print('Phase comparison')
median(Per_Transfer)
median(Per_NoTransfer)
MWc<-wilcox.test(Per_Transfer,Per_NoTransfer,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWc
qnorm(MWc$p.value/2)

[1] 39

[1] "Phase comparison"


[1] 982.4

[1] 982.4


	Wilcoxon rank sum test with continuity correction

data:  Per_Transfer and Per_NoTransfer
W = 723, p-value = 0.9917
alternative hypothesis: true location shift is not equal to 0


[1] -0.01039387

# Entropy

In [19]:
Hum_StMed <- Start_Medium$Entropy
Ag_StMed <- Start_Medium$Subject
n1<-length(Hum_StMed)
n1
Hum_StSlow <- Start_Slow$Entropy
Ag_StSlow <- Start_Slow$Subject
n2<-length(Hum_StSlow)
n2
Hum_StMedT <- Start_MediumT$Entropy
Ag_StMedT <- Start_MediumT$Subject
n1b<-length(Hum_StMedT)
n1b
Hum_StSlowT <- Start_SlowT$Entropy
Ag_StSlowT <- Start_SlowT$Subject
n2b<-length(Hum_StSlowT)
n2b
Hum_MidMed <- Mid_SMS$Entropy
Ag_MidMed <- Mid_SMS$Subject
n3<-length(Hum_MidMed)
n3
Hum_MidSlow <- Mid_MSM$Entropy
Ag_MidSlow <- Mid_MSM$Subject
n4<-length(Hum_MidSlow)
n4
Hum_MidPureM <- Mid_MMM$Entropy
Ag_MidPureM <- Mid_MMM$Subject
n5<-length(Hum_MidPureM)
n5
Hum_MidPureS <- Mid_SSS$Entropy
Ag_MidPureS <- Mid_SSS$Subject
n6<-length(Hum_MidPureS)
n6
n_Total<-n1+n2+n1b+n2b+n3+n4+n5+n6
n_Total

[1] 18

[1] 19

[1] 20

[1] 19

[1] 19

[1] 20

[1] 18

[1] 19

[1] 152

In [20]:
Speed <- c(rep("Medium",n1b),rep("Slow",n2b),rep("Medium",n1),rep("Slow",n2),rep("Medium",n3),rep("Slow",n4),rep("Medium",n5),rep("Slow",n6))
Subject <- c(Ag_StMedT,Ag_StSlowT,Ag_StMed,Ag_StSlow,Ag_MidMed,Ag_MidSlow,Ag_MidPureM,Ag_MidPureS)
Entropy <- c(Hum_StMedT,Hum_StSlowT,Hum_StMed,Hum_StSlow,Hum_MidMed,Hum_MidSlow,Hum_MidPureM,Hum_MidPureS)
Phase <- c(rep("StartT",n1b),rep("StartT",n2b),rep("StartNoT",n1),rep("StartNoT",n2),rep("MidT",n3),rep("MidT",n4),rep("MidNoT",n5),rep("MidNoT",n6))
length(Speed)
ToPlot_Ent <- data.frame(Subject,Speed,Phase,Entropy)
head(ToPlot_Ent)

[1] 152

,Subject,Speed,Phase,Entropy
,<fct>,<fct>,<fct>,<dbl>
1,A10GBO6PU2EI0K,Medium,StartT,3.246518
2,A16335MOISDG1F,Medium,StartT,3.172987
3,A1E8ZYOQSGKTP8,Medium,StartT,3.398248
4,A1YSBD5XW5UCR0,Medium,StartT,2.865347
5,A2E609OEGN4D6H,Medium,StartT,3.584057
6,A2GO3SYV3X3KE,Medium,StartT,3.345698


In [21]:
ToPlot_Ent %>% group_by(Speed,Phase) %>% shapiro_test(Entropy)

Speed,Phase,variable,statistic,p
<fct>,<fct>,<chr>,<dbl>,<dbl>
Medium,MidNoT,Entropy,0.9417047,0.309835170
Medium,MidT,Entropy,0.9291213,0.166849918
Medium,StartNoT,Entropy,0.9628566,0.657581798
Medium,StartT,Entropy,0.9632173,0.609925467
Slow,MidNoT,Entropy,0.8368486,0.004129198
Slow,MidT,Entropy,0.8996022,0.040536780
Slow,StartNoT,Entropy,0.9598709,0.569840744
Slow,StartT,Entropy,0.9541601,0.463631468


In [22]:
#A/ case Start & No Transfer
Ent_data <- ToPlot_Ent %>% filter(!Phase=='MidT') %>% filter(!Phase=='StartT')
Ent_Start <- (Ent_data %>% filter(Phase=='StartNoT'))$Entropy
Ent_Mid <- (Ent_data %>% filter(Phase=='MidNoT'))$Entropy
print('Phase comparison')
median(Ent_Start)
median(Ent_Mid)
MWa<-wilcox.test(Ent_Start,Ent_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

Ent_Slow <- (Ent_data %>% filter(Speed=='Slow'))$Entropy
Ent_Med <- (Ent_data %>% filter(Speed=='Medium'))$Entropy
print('Speed comparison')
median(Ent_Slow)
median(Ent_Med)
MWb<-wilcox.test(Ent_Med,Ent_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] 3.459006

[1] 3.350923


	Wilcoxon signed rank test with continuity correction

data:  Ent_Start and Ent_Mid
V = 564, p-value = 0.001382
alternative hypothesis: true location shift is not equal to 0


[1] -3.198293

[1] "Speed comparison"


[1] 3.489143

[1] 3.138719


	Wilcoxon rank sum test with continuity correction

data:  Ent_Med and Ent_Slow
W = 121, p-value = 1.177e-09
alternative hypothesis: true location shift is not equal to 0


[1] -6.083303

In [23]:
#B/ case Start & Transfer
Ent_data2 <- ToPlot_Ent %>% filter(!Phase=='MidNoT') %>% filter(!Phase=='StartNoT')
Ent_Start <- (Ent_data2 %>% filter(Phase=='StartT'))$Entropy
Ent_Mid <- (Ent_data2 %>% filter(Phase=='MidT'))$Entropy
print('Phase comparison')
median(Ent_Start)
median(Ent_Mid)
MWa<-wilcox.test(Ent_Start,Ent_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

Ent_Slow <- (Ent_data2 %>% filter(Speed=='Slow'))$Entropy
Ent_Med <- (Ent_data2 %>% filter(Speed=='Medium'))$Entropy
print('Speed comparison')
median(Ent_Slow)
median(Ent_Med)
MWb<-wilcox.test(Ent_Med,Ent_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] 3.329757

[1] 3.268304


	Wilcoxon signed rank test with continuity correction

data:  Ent_Start and Ent_Mid
V = 551, p-value = 0.02511
alternative hypothesis: true location shift is not equal to 0


[1] -2.239777

[1] "Speed comparison"


[1] 3.394718

[1] 3.08613


	Wilcoxon rank sum test with continuity correction

data:  Ent_Med and Ent_Slow
W = 121, p-value = 1.705e-10
alternative hypothesis: true location shift is not equal to 0


[1] -6.385771

In [24]:
#C) Transfer by comparison with No Transfer
Transfer_compare <- Ent_data2 %>% filter(Phase=='MidT')
ToAdd_Transfer <- rep('Transfer',nrow(Transfer_compare))
Transfer_compare$Group <- ToAdd_Transfer
nrow(Transfer_compare)
NoTransfer_compare <- Ent_data %>% filter(Phase=='MidNoT')
ToAdd_NoTransfer <- rep('No Transfer',nrow(NoTransfer_compare))
NoTransfer_compare$Group <- ToAdd_NoTransfer
#nrow(NoTransfer_compare)
#Compare_T <- rbind(Transfer_compare,NoTransfer_compare)
#nrow(Compare_T)

Ent_Transfer <- Transfer_compare$Entropy
Ent_NoTransfer <- NoTransfer_compare$Entropy
median(Ent_Transfer)
median(Ent_NoTransfer)
MWc<-wilcox.test(Ent_Transfer,Ent_NoTransfer,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWc
qnorm(MWc$p.value/2)

[1] 39

[1] 3.268304

[1] 3.350923


	Wilcoxon rank sum test with continuity correction

data:  Ent_Transfer and Ent_NoTransfer
W = 612, p-value = 0.2573
alternative hypothesis: true location shift is not equal to 0


[1] -1.132762

# Log CV ISI

In [25]:
Hum_StMed <- Start_Medium$lCV
Ag_StMed <- Start_Medium$Subject
n1<-length(Hum_StMed)
n1
Hum_StSlow <- Start_Slow$lCV
Ag_StSlow <- Start_Slow$Subject
n2<-length(Hum_StSlow)
n2
Hum_StMedT <- Start_MediumT$lCV
Ag_StMedT <- Start_MediumT$Subject
n1b<-length(Hum_StMedT)
n1b
Hum_StSlowT <- Start_SlowT$lCV
Ag_StSlowT <- Start_SlowT$Subject
n2b<-length(Hum_StSlowT)
n2b
Hum_MidMed <- Mid_SMS$lCV
Ag_MidMed <- Mid_SMS$Subject
n3<-length(Hum_MidMed)
n3
Hum_MidSlow <- Mid_MSM$lCV
Ag_MidSlow <- Mid_MSM$Subject
n4<-length(Hum_MidSlow)
n4
Hum_MidPureM <- Mid_MMM$lCV
Ag_MidPureM <- Mid_MMM$Subject
n5<-length(Hum_MidPureM)
n5
Hum_MidPureS <- Mid_SSS$lCV
Ag_MidPureS <- Mid_SSS$Subject
n6<-length(Hum_MidPureS)
n6
n_Total<-n1+n2+n1b+n2b+n3+n4+n5+n6
n_Total

[1] 18

[1] 19

[1] 20

[1] 19

[1] 19

[1] 20

[1] 18

[1] 19

[1] 152

In [26]:
Speed <- c(rep("Medium",n1b),rep("Slow",n2b),rep("Medium",n1),rep("Slow",n2),rep("Medium",n3),rep("Slow",n4),rep("Medium",n5),rep("Slow",n6))
Subject <- c(Ag_StMedT,Ag_StSlowT,Ag_StMed,Ag_StSlow,Ag_MidMed,Ag_MidSlow,Ag_MidPureM,Ag_MidPureS)
lCV <- c(Hum_StMedT,Hum_StSlowT,Hum_StMed,Hum_StSlow,Hum_MidMed,Hum_MidSlow,Hum_MidPureM,Hum_MidPureS)
Phase <- c(rep("StartT",n1b),rep("StartT",n2b),rep("StartNoT",n1),rep("StartNoT",n2),rep("MidT",n3),rep("MidT",n4),rep("MidNoT",n5),rep("MidNoT",n6))
length(Speed)
ToPlot_lcv <- data.frame(Subject,Speed,Phase,lCV)
head(ToPlot_lcv)

[1] 152

,Subject,Speed,Phase,lCV
,<fct>,<fct>,<fct>,<dbl>
1,A10GBO6PU2EI0K,Medium,StartT,-1.2202040
2,A16335MOISDG1F,Medium,StartT,-0.9372757
3,A1E8ZYOQSGKTP8,Medium,StartT,-0.2857093
4,A1YSBD5XW5UCR0,Medium,StartT,-1.4761271
5,A2E609OEGN4D6H,Medium,StartT,-0.4980249
6,A2GO3SYV3X3KE,Medium,StartT,-0.8536570


In [27]:
ToPlot_lcv %>% group_by(Speed,Phase) %>% shapiro_test(lCV)

Speed,Phase,variable,statistic,p
<fct>,<fct>,<chr>,<dbl>,<dbl>
Medium,MidNoT,lCV,0.9162675,1.110217e-01
Medium,MidT,lCV,0.6677649,2.352677e-05
Medium,StartNoT,lCV,0.8487408,8.090736e-03
Medium,StartT,lCV,0.8985187,3.868762e-02
Slow,MidNoT,lCV,0.7325466,1.394608e-04
Slow,MidT,lCV,0.8720865,1.278669e-02
Slow,StartNoT,lCV,0.7976930,1.058188e-03
Slow,StartT,lCV,0.7686552,4.146855e-04


In [28]:
#A/ case Start & No Transfer
lCV_data <- ToPlot_lcv %>% filter(!Phase=='MidT') %>% filter(!Phase=='StartT')
lCV_Start <- (lCV_data %>% filter(Phase=='StartNoT'))$lCV
lCV_Mid <- (lCV_data %>% filter(Phase=='MidNoT'))$lCV
print('Phase comparison')
median(lCV_Start)
median(lCV_Mid)
MWa<-wilcox.test(lCV_Start,lCV_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

lCV_Slow <- (lCV_data %>% filter(Speed=='Slow'))$lCV
lCV_Med <- (lCV_data %>% filter(Speed=='Medium'))$lCV
print('Speed comparison')
median(lCV_Slow)
median(lCV_Med)
MWb<-wilcox.test(lCV_Med,lCV_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] -1.063956

[1] -1.262679


	Wilcoxon signed rank test with continuity correction

data:  lCV_Start and lCV_Mid
V = 626, p-value = 3.571e-05
alternative hypothesis: true location shift is not equal to 0


[1] -4.133642

[1] "Speed comparison"


[1] -1.198797

[1] -1.091262


	Wilcoxon rank sum test with continuity correction

data:  lCV_Med and lCV_Slow
W = 767, p-value = 0.3723
alternative hypothesis: true location shift is not equal to 0


[1] -0.8922178

In [29]:
#B/ case Start & Transfer
lCV_data2 <- ToPlot_lcv %>% filter(!Phase=='MidNoT') %>% filter(!Phase=='StartNoT')
lCV_Start <- (lCV_data2 %>% filter(Phase=='StartT'))$lCV
lCV_Mid <- (lCV_data2 %>% filter(Phase=='MidT'))$lCV
print('Phase comparison')
median(lCV_Start)
median(lCV_Mid)
MWa<-wilcox.test(lCV_Start,lCV_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

lCV_Slow <- (lCV_data2 %>% filter(Speed=='Slow'))$lCV
lCV_Med <- (lCV_data2 %>% filter(Speed=='Medium'))$lCV
print('Speed comparison')
median(lCV_Slow)
median(lCV_Med)
MWb<-wilcox.test(lCV_Med,lCV_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] -1.168543

[1] -1.344913


	Wilcoxon signed rank test with continuity correction

data:  lCV_Start and lCV_Mid
V = 517, p-value = 0.07751
alternative hypothesis: true location shift is not equal to 0


[1] -1.765307

[1] "Speed comparison"


[1] -1.276892

[1] -1.261288


	Wilcoxon rank sum test with continuity correction

data:  lCV_Med and lCV_Slow
W = 865, p-value = 0.2987
alternative hypothesis: true location shift is not equal to 0


[1] -1.039312

In [30]:
#C) Transfer by comparison with No Transfer
Transfer_compare <- lCV_data2 %>% filter(Phase=='MidT')
ToAdd_Transfer <- rep('Transfer',nrow(Transfer_compare))
Transfer_compare$Group <- ToAdd_Transfer
nrow(Transfer_compare)
NoTransfer_compare <- lCV_data %>% filter(Phase=='MidNoT')
ToAdd_NoTransfer <- rep('No Transfer',nrow(NoTransfer_compare))
NoTransfer_compare$Group <- ToAdd_NoTransfer
#nrow(NoTransfer_compare)
#Compare_T <- rbind(Transfer_compare,NoTransfer_compare)
#nrow(Compare_T)

lCV_Transfer <- Transfer_compare$lCV
lCV_NoTransfer <- NoTransfer_compare$lCV
median(lCV_Transfer)
median(lCV_NoTransfer)
MWc<-wilcox.test(lCV_Transfer,lCV_NoTransfer,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWc
qnorm(MWc$p.value/2)

[1] 39

[1] -1.344913

[1] -1.262679


	Wilcoxon rank sum test with continuity correction

data:  lCV_Transfer and lCV_NoTransfer
W = 568, p-value = 0.1118
alternative hypothesis: true location shift is not equal to 0


[1] -1.590023

# Regularity

In [31]:
Hum_StMed <- Start_Medium$Amplitude
Ag_StMed <- Start_Medium$Subject
n1<-length(Hum_StMed)
n1
Hum_StSlow <- Start_Slow$Amplitude
Ag_StSlow <- Start_Slow$Subject
n2<-length(Hum_StSlow)
n2
Hum_StMedT <- Start_MediumT$Amplitude
Ag_StMedT <- Start_MediumT$Subject
n1b<-length(Hum_StMedT)
n1b
Hum_StSlowT <- Start_SlowT$Amplitude
Ag_StSlowT <- Start_SlowT$Subject
n2b<-length(Hum_StSlowT)
n2b
Hum_MidMed <- Mid_SMS$Amplitude
Ag_MidMed <- Mid_SMS$Subject
n3<-length(Hum_MidMed)
n3
Hum_MidSlow <- Mid_MSM$Amplitude
Ag_MidSlow <- Mid_MSM$Subject
n4<-length(Hum_MidSlow)
n4
Hum_MidPureM <- Mid_MMM$Amplitude
Ag_MidPureM <- Mid_MMM$Subject
n5<-length(Hum_MidPureM)
n5
Hum_MidPureS <- Mid_SSS$Amplitude
Ag_MidPureS <- Mid_SSS$Subject
n6<-length(Hum_MidPureS)
n6
n_Total<-n1+n2+n1b+n2b+n3+n4+n5+n6
n_Total

[1] 18

[1] 19

[1] 20

[1] 19

[1] 19

[1] 20

[1] 18

[1] 19

[1] 152

In [32]:
Speed <- c(rep("Medium",n1b),rep("Slow",n2b),rep("Medium",n1),rep("Slow",n2),rep("Medium",n3),rep("Slow",n4),rep("Medium",n5),rep("Slow",n6))
Subject <- c(Ag_StMedT,Ag_StSlowT,Ag_StMed,Ag_StSlow,Ag_MidMed,Ag_MidSlow,Ag_MidPureM,Ag_MidPureS)
Regularity <- c(Hum_StMedT,Hum_StSlowT,Hum_StMed,Hum_StSlow,Hum_MidMed,Hum_MidSlow,Hum_MidPureM,Hum_MidPureS)
Phase <- c(rep("StartT",n1b),rep("StartT",n2b),rep("StartNoT",n1),rep("StartNoT",n2),rep("MidT",n3),rep("MidT",n4),rep("MidNoT",n5),rep("MidNoT",n6))
length(Speed)
ToPlot_reg <- data.frame(Subject,Speed,Phase,Regularity)
head(ToPlot_reg)

[1] 152

,Subject,Speed,Phase,Regularity
,<fct>,<fct>,<fct>,<dbl>
1,A10GBO6PU2EI0K,Medium,StartT,0.1915128
2,A16335MOISDG1F,Medium,StartT,0.1426969
3,A1E8ZYOQSGKTP8,Medium,StartT,0.3854300
4,A1YSBD5XW5UCR0,Medium,StartT,0.4248110
5,A2E609OEGN4D6H,Medium,StartT,0.2559327
6,A2GO3SYV3X3KE,Medium,StartT,0.3034830


In [33]:
ToPlot_reg %>% group_by(Speed,Phase) %>% shapiro_test(Regularity)

Speed,Phase,variable,statistic,p
<fct>,<fct>,<chr>,<dbl>,<dbl>
Medium,MidNoT,Regularity,0.9467135,0.37581236
Medium,MidT,Regularity,0.9638838,0.65084503
Medium,StartNoT,Regularity,0.8992602,0.05577829
Medium,StartT,Regularity,0.9647867,0.64319022
Slow,MidNoT,Regularity,0.8613048,0.01030333
Slow,MidT,Regularity,0.9244852,0.12089964
Slow,StartNoT,Regularity,0.9477802,0.36203324
Slow,StartT,Regularity,0.9445918,0.31843749


In [34]:
#A/ case Start & No Transfer
Reg_data <- ToPlot_reg %>% filter(!Phase=='MidT') %>% filter(!Phase=='StartT')
Reg_Start <- (Reg_data %>% filter(Phase=='StartNoT'))$Regularity
Reg_Mid <- (Reg_data %>% filter(Phase=='MidNoT'))$Regularity
print('Phase comparison')
median(Reg_Start)
median(Reg_Mid)
MWa<-wilcox.test(Reg_Start,Reg_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

Reg_Slow <- (Reg_data %>% filter(Speed=='Slow'))$Regularity
Reg_Med <- (Reg_data %>% filter(Speed=='Medium'))$Regularity
print('Speed comparison')
median(Reg_Slow)
median(Reg_Med)
MWb<-wilcox.test(Reg_Med,Reg_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] 0.1539936

[1] 0.1817721


	Wilcoxon signed rank test with continuity correction

data:  Reg_Start and Reg_Mid
V = 114, p-value = 0.0003496
alternative hypothesis: true location shift is not equal to 0


[1] -3.57545

[1] "Speed comparison"


[1] 0.1286242

[1] 0.1979483


	Wilcoxon rank sum test with continuity correction

data:  Reg_Med and Reg_Slow
W = 1041, p-value = 0.0001155
alternative hypothesis: true location shift is not equal to 0


[1] -3.855462

In [35]:
#B/ case Start & Transfer
Reg_data2 <- ToPlot_reg %>% filter(!Phase=='MidNoT') %>% filter(!Phase=='StartNoT')
Reg_Start <- (Reg_data2 %>% filter(Phase=='StartT'))$Regularity
Reg_Mid <- (Reg_data2 %>% filter(Phase=='MidT'))$Regularity
print('Phase comparison')
median(Reg_Start)
median(Reg_Mid)
MWa<-wilcox.test(Reg_Start,Reg_Mid,paired=TRUE,conf.level = 0.95,exact=FALSE)
MWa
qnorm(MWa$p.value/2)

Reg_Slow <- (Reg_data2 %>% filter(Speed=='Slow'))$Regularity
Reg_Med <- (Reg_data2 %>% filter(Speed=='Medium'))$Regularity
print('Speed comparison')
median(Reg_Slow)
median(Reg_Med)
MWb<-wilcox.test(Reg_Med,Reg_Slow,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWb
qnorm(MWb$p.value/2)

[1] "Phase comparison"


[1] 0.1915128

[1] 0.2067543


	Wilcoxon signed rank test with continuity correction

data:  Reg_Start and Reg_Mid
V = 334, p-value = 0.4386
alternative hypothesis: true location shift is not equal to 0


[1] -0.7745024

[1] "Speed comparison"


[1] 0.1323817

[1] 0.2550804


	Wilcoxon rank sum test with continuity correction

data:  Reg_Med and Reg_Slow
W = 1300, p-value = 7.187e-08
alternative hypothesis: true location shift is not equal to 0


[1] -5.386433

In [36]:
#C) Transfer by comparison with No Transfer
Transfer_compare <- Reg_data2 %>% filter(Phase=='MidT')
ToAdd_Transfer <- rep('Transfer',nrow(Transfer_compare))
Transfer_compare$Group <- ToAdd_Transfer
nrow(Transfer_compare)
NoTransfer_compare <- Reg_data %>% filter(Phase=='MidNoT')
ToAdd_NoTransfer <- rep('No Transfer',nrow(NoTransfer_compare))
NoTransfer_compare$Group <- ToAdd_NoTransfer
#nrow(NoTransfer_compare)
#Compare_T <- rbind(Transfer_compare,NoTransfer_compare)
#nrow(Compare_T)

Reg_Transfer <- Transfer_compare$Regularity
Reg_NoTransfer <- NoTransfer_compare$Regularity
median(Reg_Transfer)
median(Reg_NoTransfer)
MWc<-wilcox.test(Reg_Transfer,Reg_NoTransfer,paired=FALSE,conf.level = 0.95,exact=FALSE)
MWc
qnorm(MWc$p.value/2)

[1] 39

[1] 0.2067543

[1] 0.1817721


	Wilcoxon rank sum test with continuity correction

data:  Reg_Transfer and Reg_NoTransfer
W = 755, p-value = 0.7316
alternative hypothesis: true location shift is not equal to 0


[1] -0.3429462